In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import json
import seaborn as sns
sns.set()
%matplotlib inline

## Step1: build the initial state of the entire user network, as well as the purchae history of the users
Input: sample_dataset/batch_log.json

In [2]:
batchlogfile = 'sample_dataset/batch_log.json'
df = pd.read_json(batchlogfile, lines=True)

In [3]:
df.head(10)

,D,T,amount,event_type,id,id1,id2,timestamp
0,2.0,50.0,NaN,NaN,NaN,NaN,NaN,NaT
1,NaN,NaN,56.69,purchase,250.0,NaN,NaN,2017-06-14 18:46:40
2,NaN,NaN,20.68,purchase,8921.0,NaN,NaN,2017-06-14 18:46:40
3,NaN,NaN,27.91,purchase,5053.0,NaN,NaN,2017-06-14 18:46:40
4,NaN,NaN,76.61,purchase,2204.0,NaN,NaN,2017-06-14 18:46:40
5,NaN,NaN,NaN,befriend,NaN,6981.0,3402.0,2017-06-14 18:46:40
6,NaN,NaN,56.01,purchase,9572.0,NaN,NaN,2017-06-14 18:46:40
7,NaN,NaN,NaN,befriend,NaN,6037.0,8071.0,2017-06-14 18:46:40
8,NaN,NaN,73.72,purchase,5362.0,NaN,NaN,2017-06-14 18:46:40
9,NaN,NaN,NaN,befriend,NaN,6185.0,8617.0,2017-06-14 18:46:40


In [4]:
df.describe()

,D,T,amount,id,id1,id2
count,1.0,1.0,400257.000000,400257.000000,99742.000000,99742.000000
mean,2.0,50.0,66.174229,4996.314258,5006.222544,5012.469762
std,NaN,NaN,43.120066,2884.169683,2881.856962,2889.195433
min,2.0,50.0,1.000000,0.000000,0.000000,0.000000
25%,2.0,50.0,32.120000,2498.000000,2517.000000,2513.000000
50%,2.0,50.0,57.970000,5003.000000,5003.000000,5018.000000
75%,2.0,50.0,92.710000,7488.000000,7497.000000,7520.000000
max,2.0,50.0,318.670000,9999.000000,9999.000000,9999.000000


In [5]:
# Read D and T
df_DT=df[df['D'].notnull()]
df_DT=df_DT[['D','T']]
D = df_DT.values[0][0]
T = df_DT.values[0][1]
print(D)
print(T)
df_DT.head()

2.0
50.0


,D,T
0,2.0,50.0


In [6]:
# check D and T values
if D < 1:
    print('Program terminated because of D < 1')
    sys.exit()
if T < 2:
    print('Program terminated because of T < 2')
    sys.exit()

In [7]:
#for possible_value in set(df['event_type'].tolist()):
#    print(possible_value)

In [8]:
df_purchase = df[df['event_type']=='purchase']
df_purchase = df_purchase[['event_type','id','timestamp','amount']]
df_purchase = df_purchase.sort_values('timestamp')
df_purchase.shape

(400257, 4)

In [9]:
df_friend=df[(df['event_type']=='befriend') | (df['event_type']=='unfriend')]
df_friend=df_friend[['event_type','id1','id2','timestamp']]
df_friend=df_friend.sort_values('timestamp')
df_friend.shape

(99742, 4)

In [10]:
G = nx.Graph()

In [11]:
id1list = df_friend.id1.tolist()
id2list = df_friend.id2.tolist()
idlist = set(id1list + id2list)
G.add_nodes_from(idlist)
len(list(G.nodes()))

10000

In [12]:
def Add_edges(data):
    for i in range(0, len(data)):
        datai = data.iloc[i]
        id10 = datai['id1']
        id20 = datai['id2']
        event_type0 = datai['event_type']
        if event_type0 == 'befriend':
            G.add_edge(id10,id20)
        if event_type0 == 'unfriend':
            if G.has_edge(id10,id20):
                G.remove_edge(id10,id20)  
                            

In [13]:
Add_edges(df_friend)

In [14]:
len(list(G.edges()))

90460

In [15]:
nnode = 1.0

df_friend[df_friend['id1'] == nnode]

,event_type,id1,id2,timestamp
16950,befriend,1.0,1957.0,2017-06-14 18:46:40
54199,befriend,1.0,6374.0,2017-06-14 18:46:41
57310,befriend,1.0,7836.0,2017-06-14 18:46:41
136571,befriend,1.0,5977.0,2017-06-14 18:46:42
194384,befriend,1.0,6743.0,2017-06-14 18:46:44
249670,befriend,1.0,7772.0,2017-06-14 18:46:45
391922,befriend,1.0,7270.0,2017-06-14 18:46:47
496952,befriend,1.0,8914.0,2017-06-14 18:46:49
469129,befriend,1.0,3690.0,2017-06-14 18:46:49


In [16]:
#G[10.0]

In [17]:
G.number_of_nodes()

10000

In [18]:
G.number_of_edges()

90460

In [19]:
#list(G.edges())

In [20]:
#list(G.neighbors(nnode))

In [21]:
#list(G.neighbors(3.0))

In [22]:
#nnode = 1.0

Dlist = list(G.neighbors(nnode))
#print(Dlist)
for i in list(G.neighbors(nnode)):
    #print(list(G.neighbors(i)))
    Dlist += list(G.neighbors(i))
    #print(Dlist)
    #print(len(list(G.neighbors(i))))

#print(Dlist)
#print(set(Dlist))
print(len(set(Dlist)))



327


In [23]:
len(list(nx.ego_graph(G, nnode, D, center=False)))

326

In [24]:
# define a dataframe: df_mean_sd to store the mean and sd of the last T purchase for each user's network
#df_mean_sd = pd.DataFrame({"id": [], "mean": [], "sd": []})
#df_mean_sd

In [25]:
# define a function to calcualte the mean and sd for userid's network
def Get_Mean_SD(userid):
    Nodes = list(nx.ego_graph(G, userid, D, center=False))
    df_Nodes = df_purchase.loc[df_purchase['id'].isin(Nodes)]
    if len(df_Nodes) >= 2:    
        if len(df_Nodes) > T:
            df_Nodes = df_Nodes.sort_values('timestamp').iloc[-int(T):]
        df_Nodes.shape
        mean = df_Nodes.amount.mean()
        sd = df_Nodes.amount.std()
        mean = float("{0:.2f}".format(mean))
        sd = float("{0:.2f}".format(sd))
    #Node_amount = pd.DataFrame({'id': [userid], 'mean': [mean], 'sd': [sd]})
    #global df_mean_sd
    #df_mean_sd = df_mean_sd.append(Node_amount)
    else:
        mean=np.nan
        sd=np.nan
    
    return mean, sd

In [26]:
#Get_Mean_SD(4.0)

In [27]:
#df_mean_sd

## Step2: Determine where a purchase is anomalous 
input file: sample_dataset/stream_log.json

In [28]:
# read in the stream_log.json
streamlogfile = 'sample_dataset/stream_log.json'
df_stream = pd.read_json(streamlogfile, lines=True)
# If sort on the timestamp is needed, commentout the following line
#df_stream = df_stream.sort_values('timestamp')

# open output file flagged_purchases.json
#flaggedfile = 'log_output/flagged_purchases.json'
flaggedfile = 'flagged_purchases.json'
f = open(flaggedfile, 'a+')

In [29]:
# Determine a purchase is anomalous; update purchase history; update social network
for i in range(0, len(df_stream)):
    datai = df_stream.iloc[i]
    event_type = datai['event_type']
    if event_type == 'purchase':
        # update purchase history
        df_purchase = df_purchase.append(datai)
        timestamp = datai['timestamp']
        timestamp = str(timestamp)
        userid = datai['id']
        amount = datai['amount']
        mean, sd = Get_Mean_SD(userid)
        if mean != np.nan:
            mean_3sd = mean + (3*sd)
            dataflag = {"event_type":event_type, "timestamp":str(timestamp), "id": userid, "amount": amount, "mean": mean, "sd": sd}
            if amount > mean_3sd:
                f.write('"event_type":"{0:s}", "timestamp":"{1:s}", "id": "{2:.0f}", "amount": "{3:.2f}", "mean": "{4:.2f}", "sd": "{5:.2f}"\n'.format(event_type, timestamp, userid, amount, mean, sd))
    # update social network
    if event_type == 'befriend':
        id1 = datai['id1']
        id2 = datai['id2']
        G.add_edge(id1,id2)
    if event_type == 'unfriend':
        id1 = datai['id1']
        id2 = datai['id2']
        if G.has_edge(id1,id2):
            G.remove_edge(id1,id2)  
    

In [30]:
f.close() 